# Implementacja "Robust global stabilization and disturbance rejection of an underactuated nonholonomic airship"

Link do implementowanego artykułu: https://ieeexplore.ieee.org/document/4389345

Imports

In [1]:
import sympy as sp
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt # ploting results
from scipy.integrate import solve_ivp # solving differential equations
MAT = np.array

Definicja zmiennych  - rozdział II

In [2]:
t = sp.symbols('t') # czas
th = sp.symbols('\\theta') # kąt obrotu sterowca w układzie globalnym
u = sp.symbols('u') # prędkość wzdłużna sterowca w układzie lokalnym
v = sp.symbols('v') # prędkość poprzeczna sterowca w układzie lokalnym
r = sp.symbols('r') # prędkość obrotowa sterowca w układzie globalnym
x, y = sp.symbols('x y') # położenie sterowca w układzie globalnym
z1, z2, z3 = sp.symbols('z_{1} z_{2} z_{3}') # ?
tauu, taur = sp.symbols('\\tau_{u} \\tau_{r}') # sygnały sterujące kolejno: siła ciągu, siła skręcająca 
ub, vb, z1b, z2b, z3b = sp.symbols('\\bar{u} \\bar{v} \\bar{z}_{1} \\bar{z}_{2} \\bar{z}_{3}')#nowe zmienne stanu po transforma

Definicja współczynników i stałych 

In [3]:
Cu, Cv, Cr = sp.symbols('C_{u} C_{v} C_{r}') #współczynniki aerodynamiczne względem osi u v r
Vw = sp.symbols('V_{w}') #prędkość wiatru (w artykule jest zawsze stała)
a_ship, b_ship = sp.symbols('a_{ship} b_{ship}') #długość sterowca, szerokość sterowca
m, m3 = sp.symbols('m m_{3}') # masa strowca m=m1=m2, masa sterowca m3
d1, d2, d3 = sp.symbols('d_{1} d_{2} d_{3}') # współczynniki tłumienia kolejno w osi u v r
d1f, d2f, d3f, th3f = sp.symbols('\\underline{d}_{1} \\underline{d}_{2} \\underline{d}_{3} \\underline{\\theta}_{3}') # dolne wartości wsp tłumienia d1, d2, d3
d1c, d2c, d3c, th3c = sp.symbols('\\bar{d}_{1} \\bar{d}_{2} \\bar{d}_{3} \\bar{\\theta}_{3}') # górne wartości współczynników tłumienia d1, d2, d3
d10, d20, d30, th30 = sp.symbols('d_{10} d_{20} d_{30} \\theta_{30}') # niepewności
Delta_1, Delta_2, Delta_3, delta_1, delta_2, delta_3 = sp.symbols('\\Delta_{1} \\Delta_{2} \\Delta_{3} \\delta_{1} \\delta_{2} \\delta_{3}')# niepewności
k1, k2, k3, a = sp.symbols('k_{1} k_{2} k_{3} a') # współczynniki wzmocnień syg ster tau_r
k_th, k_r  = sp.symbols('k_{\\theta} k_{r}') # jak wyżej
k1b, k2b, k3b, k4b, k5b = sp.symbols('\\bar{k}_{1} \\bar{k}_{2} \\bar{k}_{3} \\bar{k}_{4} \\bar{k}_{5}')#wsp wzm sterownika tau_u
th_1, th_2, th_3 = sp.symbols('\\theta_1 \\theta_2 \\theta_3') # współczynniki th z obliczone na podst wzoru 4.4
lam_1, lam_2 = sp.symbols('\lambda_1 \lambda_2') # współczynniki lam z obliczone na podst wzoru 4.4
delta_d = sp.symbols('\\Delta_{d}') # współczynnik deltad ze wzoru 4.3
th_0, r_0 = sp.symbols('th(0) r(0)')# wartości początkowe

Definicja funkcji - rozdział II (2.2)

In [4]:
def F(angle):
    return sp.sqrt(a_ship**2 * sp.sin(angle)**2 + b_ship**2 * sp.cos(angle)**2)
def fu(angle):
    return Cu * Vw**2 * F(angle) * sp.cos(angle)
def fv(angle):
    return Cv * Vw**2 * F(angle) * sp.sin(angle)
def fr(angle):
    return Cr * Vw**2 * F(angle) * sp.cos(2 * angle)
Ff = sp.symbols('F', cls=sp.Function)
#wyświetlanie implementowanych wzorów
display(fu(th),fv(th),fr(th),F(th))

C_{u}*V_{w}**2*sqrt(a_{ship}**2*sin(\theta)**2 + b_{ship}**2*cos(\theta)**2)*cos(\theta)

C_{v}*V_{w}**2*sqrt(a_{ship}**2*sin(\theta)**2 + b_{ship}**2*cos(\theta)**2)*sin(\theta)

C_{r}*V_{w}**2*sqrt(a_{ship}**2*sin(\theta)**2 + b_{ship}**2*cos(\theta)**2)*cos(2*\theta)

sqrt(a_{ship}**2*sin(\theta)**2 + b_{ship}**2*cos(\theta)**2)

Opis modelu kinematycznego, geometrycznego powiązania pomiędzy układem globalnym a układem lokalnym - rozdział II (2.4)

In [5]:
def Kinematyka(u_,v_,r_,angle):
    dx = u_ * sp.cos(angle) - v_ * sp.sin(angle)
    dy = u_ * sp.sin(angle) + v_ * sp.cos(angle)
    dth = r_
    return sp.Matrix([[dx,dy,dth]]).T
#wyświetlanie implementowanych wzorów
#display(Kinematyka(u,v,r,th))

Ciąg dalszy opisu kinematyki, z1 z2 - rozdział II (2.5)

In [6]:
def Z12(x_,y_,angle):
    z1 = x_ * sp.cos(angle) + y_ * sp.sin(angle)
    z2 = -x_ * sp.sin(angle) + y_ * sp.cos(angle)
    return sp.Matrix([[z1,z2]]).T
#wyświetlanie implementowanych wzorów
#display(Z12(x,y,th))

Równanie stanu uzyskane przez połączenie kinematyki i dynamiki sterowca - rozdział II (2.6)

In [7]:
# układ Sigma1:
# macierz stanu
def A1(r_):
    return sp.Matrix([[-d1,r_,0,0],[-r_,-d2,0,0],[1,0,0,r_],[0,1,-r_,0]]) 
#---------
q = sp.Matrix([[u,v,z1,z2]]).T # wektor stanu
b1 = sp.Matrix([[1/m,0,0,0]]).T # macierz sterowania
# wektor sił zakłóceń - wiatru
def f(angle):
    return sp.Matrix([[fu(angle),fv(angle),0,0]]).T 
#system dynamiczny
def dQ1(q_,r_,tauu_,ang):
    return A1(r_) @ q_ + b1 * tauu_ + f(ang)/m
#wyświetlanie implementowanych wzorów
display(A1(r),q,b1,f(th),dQ1(q,r,tauu,th))

Matrix([
[-d_{1},      r,  0, 0],
[    -r, -d_{2},  0, 0],
[     1,      0,  0, r],
[     0,      1, -r, 0]])

Matrix([
[    u],
[    v],
[z_{1}],
[z_{2}]])

Matrix([
[1/m],
[  0],
[  0],
[  0]])

Matrix([
[C_{u}*V_{w}**2*sqrt(a_{ship}**2*sin(\theta)**2 + b_{ship}**2*cos(\theta)**2)*cos(\theta)],
[C_{v}*V_{w}**2*sqrt(a_{ship}**2*sin(\theta)**2 + b_{ship}**2*cos(\theta)**2)*sin(\theta)],
[                                                                                       0],
[                                                                                       0]])

Matrix([
[C_{u}*V_{w}**2*sqrt(a_{ship}**2*sin(\theta)**2 + b_{ship}**2*cos(\theta)**2)*cos(\theta)/m + \tau_{u}/m - d_{1}*u + r*v],
[             C_{v}*V_{w}**2*sqrt(a_{ship}**2*sin(\theta)**2 + b_{ship}**2*cos(\theta)**2)*sin(\theta)/m - d_{2}*v - r*u],
[                                                                                                            r*z_{2} + u],
[                                                                                                           -r*z_{1} + v]])

In [8]:
# układ Sigma2:
def dQ2(r_,taur_,angle):
    dr = -d3 * r_ + (1/m3) * (taur_ + fr(angle))
    dth = r_
    return sp.Matrix([[dr,dth]]).T

#wyświetlanie implementowanych wzorów
display(dQ2(r,taur,th))

Matrix([
[-d_{3}*r + (C_{r}*V_{w}**2*sqrt(a_{ship}**2*sin(\theta)**2 + b_{ship}**2*cos(\theta)**2)*cos(2*\theta) + \tau_{r})/m_{3}],
[                                                                                                                       r]])

Sygnał sterujący dla podsystemu Sigma2 - rozdział IV (4.1)

In [9]:
def Ctrl_r(t_,r_,angle):
    return m3 * (-k_r * r_ - k_th * angle + a * sp.exp(-k3 * t_) - fr(angle))
#wyświetlanie implementowanych wzorów
display(Ctrl_r(t,r,th))
k_th = k1 * k2
k_r = k1 + k2 - d3f

m_{3}*(-C_{r}*V_{w}**2*sqrt(a_{ship}**2*sin(\theta)**2 + b_{ship}**2*cos(\theta)**2)*cos(2*\theta) - \theta*k_{\theta} + a*exp(-k_{3}*t) - k_{r}*r)

Odpowiedź kąta i prędkości kątowej jest dana - rozdział IV (4.3)

In [10]:
def Th(t_):
    return th_1*sp.exp(-lam_1*t_) + th_2*sp.exp(-lam_2*t_) + th_3*sp.exp(-k3*t_) # th(t)
def R(t_):
    return -th_1*lam_1*sp.exp(-lam_1*t_) - th_2*lam_2*sp.exp(-lam_2*t_) - th_3*k3*sp.exp(-k3*t_) # r(t)
tta, rf = sp.symbols('\\theta r', cls=sp.Function) # to będą później funkcje te powyżej
#wyświetlanie implementowanych wzorów
display(tta(t),Th(t),rf(t),R(t))

\theta(t)

\theta_1*exp(-\lambda_1*t) + \theta_2*exp(-\lambda_2*t) + \theta_3*exp(-k_{3}*t)

r(t)

-\lambda_1*\theta_1*exp(-\lambda_1*t) - \lambda_2*\theta_2*exp(-\lambda_2*t) - \theta_3*k_{3}*exp(-k_{3}*t)

Implementacja współczynników theta - rozdział IV (4.4)

In [11]:
th_1 = ((lam_2*th_0 + r_0)/(lam_2 - lam_1)) + a/((lam_2 - lam_1)*(k3 - lam_1))
th_2 = ((k1*th_0 + r_0)/(lam_1 - lam_2)) + a/((lam_1 - lam_2)*(k3 - lam_2))
th_3 = a/((lam_1 - k3)*(lam_2 - k3))
#wyświetlanie implementowanych wzorów
display(th_1,th_2,th_3)

a/((-\lambda_1 + \lambda_2)*(-\lambda_1 + k_{3})) + (\lambda_2*th(0) + r(0))/(-\lambda_1 + \lambda_2)

a/((\lambda_1 - \lambda_2)*(-\lambda_2 + k_{3})) + (k_{1}*th(0) + r(0))/(\lambda_1 - \lambda_2)

a/((\lambda_1 - k_{3})*(\lambda_2 - k_{3}))

Implementacja współczynników lambda - rozdział IV (4.4)

In [12]:
lam_1 = ((delta_d + k1 + k2) + sp.sqrt(delta_d**2 + 2*delta_d*(k1 + k2) + (k1 - k2)**2))/2
lam_2 = ((delta_d + k1 + k2) - sp.sqrt(delta_d**2 + 2*delta_d*(k1 + k2) + (k1 - k2)**2))/2
delta_d = d3 - d3f
#wyświetlanie implementowanych wzorów
display(lam_1,lam_2,delta_d)

\Delta_{d}/2 + k_{1}/2 + k_{2}/2 + sqrt(\Delta_{d}**2 + 2*\Delta_{d}*(k_{1} + k_{2}) + (k_{1} - k_{2})**2)/2

\Delta_{d}/2 + k_{1}/2 + k_{2}/2 - sqrt(\Delta_{d}**2 + 2*\Delta_{d}*(k_{1} + k_{2}) + (k_{1} - k_{2})**2)/2

-\underline{d}_{3} + d_{3}

Implementacja systemu Sigma1e - rozdział IV (4.7)

In [13]:
def A2(t_):
    return sp.Matrix([[-d1,rf(t_),0,0,0],[-rf(t_),-d2,0,0,0],[1,0,0,rf(t_),0],[0,1,-rf(t_),0,0],[0,0,1,0,0]])# macierz stanu
b2 = sp.Matrix([[1/m,0,0,0,0]]).T # macierz sterowania
def ff(angle):
    return sp.Matrix([[fu(angle),-fv(angle),0,0,0]]).T # wektor zakłóceń - wiatru
q2 = sp.Matrix([[u,v,z1,z2,z3]]).T # wektor stanu
def dX1(q2_,angle,tauu_,t_):
    return A2(t_) @ q2_ + b2 * tauu_ + ff(angle)/m
#wyświetlanie implementowanych wzorów
display(A2(t),b2,ff(th),q2,dX1(q2,th,tauu,t))

Matrix([
[-d_{1},   r(t),     0,    0, 0],
[ -r(t), -d_{2},     0,    0, 0],
[     1,      0,     0, r(t), 0],
[     0,      1, -r(t),    0, 0],
[     0,      0,     1,    0, 0]])

Matrix([
[1/m],
[  0],
[  0],
[  0],
[  0]])

Matrix([
[ C_{u}*V_{w}**2*sqrt(a_{ship}**2*sin(\theta)**2 + b_{ship}**2*cos(\theta)**2)*cos(\theta)],
[-C_{v}*V_{w}**2*sqrt(a_{ship}**2*sin(\theta)**2 + b_{ship}**2*cos(\theta)**2)*sin(\theta)],
[                                                                                        0],
[                                                                                        0],
[                                                                                        0]])

Matrix([
[    u],
[    v],
[z_{1}],
[z_{2}],
[z_{3}]])

Matrix([
[C_{u}*V_{w}**2*sqrt(a_{ship}**2*sin(\theta)**2 + b_{ship}**2*cos(\theta)**2)*cos(\theta)/m + \tau_{u}/m - d_{1}*u + v*r(t)],
[            -C_{v}*V_{w}**2*sqrt(a_{ship}**2*sin(\theta)**2 + b_{ship}**2*cos(\theta)**2)*sin(\theta)/m - d_{2}*v - u*r(t)],
[                                                                                                            u + z_{2}*r(t)],
[                                                                                                            v - z_{1}*r(t)],
[                                                                                                                     z_{1}]])

Wektor stanu po zastosowaniu gładkiej transformacji współrzędnych - rozdział IV (4.9)

In [14]:
ub = u
vb = v * sp.exp(k3 * t)
z1b = z1
z2b = z2 * sp.exp(k3 * t)
z3b = z3

Nowy wektor stanu - rozdział IV (4.10)

In [15]:
Xb = sp.Matrix([[ub,vb,z1b,z2b,z3b]]).T
#wyświetlanie implementowanych wzorów
#display(Xb)

Współczynniki d1, d2, th3 - rozdział IV (4.11)

In [16]:
d1 = d10 + Delta_1
d2 = d20 + Delta_2
d3 = d30 + Delta_3#
th3 = th30 + Delta_3
#wyświetlanie implementowanych wzorów
display(d1,d2,d3,th3)

\Delta_{1} + d_{10}

\Delta_{2} + d_{20}

\Delta_{3} + d_{30}

\Delta_{3} + \theta_{30}

Wzory potrzebne do obliczenia d1, d2, th3 - rozdział IV (4.12)

In [17]:
th3f = a/((k3 - k1)*(k3 - k2))
th3c = a/(k3**2 - (d3c + k_r)*k3 + k1*k2)
d10 = (d1c + d1f)/2
d20 = (d2c + d2f)/2
d30 = (d3c + d3f)/2
th30 = (th3c + th3f)/2
delta_1 = (d1c - d1f)/2
delta_2 = (d2c - d2f)/2
delta_3 = (th3c - th3f)/2
#wyświetlanie implementowanych wzorów
display(th3f,th3c,d10,d20,d30,th30,delta_1,delta_2,delta_3)

a/((-k_{1} + k_{3})*(-k_{2} + k_{3}))

a/(k_{1}*k_{2} + k_{3}**2 - k_{3}*(\bar{d}_{3} - \underline{d}_{3} + k_{1} + k_{2}))

\bar{d}_{1}/2 + \underline{d}_{1}/2

\bar{d}_{2}/2 + \underline{d}_{2}/2

\bar{d}_{3}/2 + \underline{d}_{3}/2

a/(2*(k_{1}*k_{2} + k_{3}**2 - k_{3}*(\bar{d}_{3} - \underline{d}_{3} + k_{1} + k_{2}))) + a/(2*(-k_{1} + k_{3})*(-k_{2} + k_{3}))

\bar{d}_{1}/2 - \underline{d}_{1}/2

\bar{d}_{2}/2 - \underline{d}_{2}/2

a/(2*(k_{1}*k_{2} + k_{3}**2 - k_{3}*(\bar{d}_{3} - \underline{d}_{3} + k_{1} + k_{2}))) - a/(2*(-k_{1} + k_{3})*(-k_{2} + k_{3}))

Rozszerzony system Sigma1e bez zakłócenia f(t) - rozdział IV (4.13) i (4.14)

In [18]:
#macierze o stałych współczynnikach
def A10():
    return sp.Matrix([[-d10,0,0,0,0],[th30*k3,-(d20-k3),0,0,0],[1,0,0,0,0],[0,1,th30*k3,k3,0],[0,0,1,0,0]])
def D():
    return sp.Matrix([[1,0,0,0],[0,1,1,0],[0,0,0,0],[0,0,0,1],[0,0,0,0]])
def E():
    return sp.Matrix([[-delta_1,0,0,0,0],[0,-delta_2,0,0,0],[k3*delta_3,0,0,0,0],[0,0,k3*delta_3,0,0]])
def DELTA():
    return sp.Matrix.diag([Delta_1/delta_1,Delta_2/delta_2,Delta_3/delta_3,Delta_3/delta_3])
#funkcja alpha(t)
def Alp(t_):
    return -(th_1*lam_1*sp.exp(-(lam_1-k3)*t_)+th_2*lam_2*sp.exp(-(lam_2-k3)*t_))
af = sp.symbols('\\alpha', cls=sp.Function)# to będzie później funkcja alpha(t)
#macierz o zmiennych współczynnikach
def A22(t_):
    return sp.Matrix([[0,rf(t_)*sp.exp(-k3*t_),0,0,0],[af(t_),0,0,0,0],[0,0,0,rf(t_)*sp.exp(-k3*t_),0],[0,0,af(t_),0,0],[0,0,0,0,0]])
#wyświetlanie implementowanych wzorów
#display(Alp(t),A22(t))
display(A10(),D(),E(),DELTA())

Matrix([
[                                                                                                      -\bar{d}_{1}/2 - \underline{d}_{1}/2,                                            0,                                                                                                                                          0,     0, 0],
[k_{3}*(a/(2*(k_{1}*k_{2} + k_{3}**2 - k_{3}*(\bar{d}_{3} - \underline{d}_{3} + k_{1} + k_{2}))) + a/(2*(-k_{1} + k_{3})*(-k_{2} + k_{3}))), -\bar{d}_{2}/2 - \underline{d}_{2}/2 + k_{3},                                                                                                                                          0,     0, 0],
[                                                                                                                                         1,                                            0,                                                                                                                                 

Matrix([
[1, 0, 0, 0],
[0, 1, 1, 0],
[0, 0, 0, 0],
[0, 0, 0, 1],
[0, 0, 0, 0]])

Matrix([
[                                                                                                      -\bar{d}_{1}/2 + \underline{d}_{1}/2,                                    0,                                                                                                                                          0, 0, 0],
[                                                                                                                                         0, -\bar{d}_{2}/2 + \underline{d}_{2}/2,                                                                                                                                          0, 0, 0],
[k_{3}*(a/(2*(k_{1}*k_{2} + k_{3}**2 - k_{3}*(\bar{d}_{3} - \underline{d}_{3} + k_{1} + k_{2}))) - a/(2*(-k_{1} + k_{3})*(-k_{2} + k_{3}))),                                    0,                                                                                                                                          0, 0, 0],
[            

Matrix([
[\Delta_{1}/(\bar{d}_{1}/2 - \underline{d}_{1}/2),                                                0,                                                                                                                                               0,                                                                                                                                               0],
[                                               0, \Delta_{2}/(\bar{d}_{2}/2 - \underline{d}_{2}/2),                                                                                                                                               0,                                                                                                                                               0],
[                                               0,                                                0, \Delta_{3}/(a/(2*(k_{1}*k_{2} + k_{3}**2 - k_{3}*(\bar{d}_{3} - \underline{d}_{3} + k_{1} + k_{2}))) - a/(

Sygnał sterujący tau_u - rozdział IV (4.21)

In [19]:
def Ctrl_u(ub_,vb_,z1b_,z2b_,z3b_):
    return -(k1b*ub_+k2b*vb_+k3b*z1b_+k4b*z2b_+k5b*z3b_)
#wyświetlanie implementowanych wzorów
display(Ctrl_u(ub,vb,z1b,z2b,z3b))

-\bar{k}_{1}*u - \bar{k}_{2}*v*exp(k_{3}*t) - \bar{k}_{3}*z_{1} - \bar{k}_{4}*z_{2}*exp(k_{3}*t) - \bar{k}_{5}*z_{3}

Implementacja systemu - rozdział IV (4.25)

In [20]:
kb = sp.Matrix([[k1b,k2b,k3b,k4b,k5b]]) #wektor wzmocnień sterownika
def Ac():
    kb_ = sp.Matrix([[k1b,k2b,k3b,k4b,k5b]])
    return A10() - b2@kb_ + D()@DELTA()@E()
#wektor zakłóceń
def fb(t_):
    return sp.Matrix([[Cu*Vw**2*Ff(tta(t_))*sp.cos(tta(t_)),-Cv*Vw**2*sp.exp(k3*t_)*Ff(tta(t_))*sp.sin(tta(t_)),0,0,0]]).T
#równanie systemu Sigma1e po dekompozycji na macierz stałą i zależną od czasu
def dXb(Xb_,t_):
    return (Ac() + A2(t_))@Xb_ + fb(t_)/m
#wyświetlanie implementowanych wzorów
display(kb,Ac(),fb(t))
#display(dXb(Xb,t))

Matrix([[\bar{k}_{1}, \bar{k}_{2}, \bar{k}_{3}, \bar{k}_{4}, \bar{k}_{5}]])

Matrix([
[                \Delta_{1}*(-\bar{d}_{1}/2 + \underline{d}_{1}/2)/(\bar{d}_{1}/2 - \underline{d}_{1}/2) - \bar{d}_{1}/2 - \bar{k}_{1}/m - \underline{d}_{1}/2,                                                                                                                        -\bar{k}_{2}/m,                                                                                                                                                -\bar{k}_{3}/m, -\bar{k}_{4}/m, -\bar{k}_{5}/m],
[\Delta_{3}*k_{3} + k_{3}*(a/(2*(k_{1}*k_{2} + k_{3}**2 - k_{3}*(\bar{d}_{3} - \underline{d}_{3} + k_{1} + k_{2}))) + a/(2*(-k_{1} + k_{3})*(-k_{2} + k_{3}))), \Delta_{2}*(-\bar{d}_{2}/2 + \underline{d}_{2}/2)/(\bar{d}_{2}/2 - \underline{d}_{2}/2) - \bar{d}_{2}/2 - \underline{d}_{2}/2 + k_{3},                                                                                                                                                             0,              0,              0],
[                

Matrix([
[              C_{u}*V_{w}**2*F(\theta(t))*cos(\theta(t))],
[-C_{v}*V_{w}**2*F(\theta(t))*exp(k_{3}*t)*sin(\theta(t))],
[                                                       0],
[                                                       0],
[                                                       0]])

# Podstawienie wartości do zmiennych symbolicznych

In [21]:
params = {a_ship:1.2,b_ship:0.5} #Podstawienie wymiarów sterowca
params.update({Cu:0.42,Cv:0.42,Cr:0.42,Vw:0.1}) #Podstawienie wartości współczynników aerodynamicznych i prędkości wiatru
params.update({m:0.072, m3:0.018})#Podstawienie wartości masy strowca m=m1=m2, masa sterowca m3
params.update({d1f:0.008, d2f:0.029, d3f:0.035})#Podstawienie dolnych wartości współczynników tłumienia
params.update({d1c:0.01, d2c:0.057, d3c:0.05})#Podstawienie górnych wartości współczynników tłumienia
params.update({k1:0.52, k2:0.5, k3:0.35, a:0.15})#Podstawienie wartości wzmocnień kontrolera tau_r
params.update({k1b:1.6, k2b:0.13, k3b:1.0, k4b:0.16, k5b:0.0032})#Podstawienie wartości
params.update({})#Podstawienie wartości
params.update({})#Podstawienie wartości
params.update({})#Podstawienie wartości
params.update({})#Podstawienie wartości(tych wart nie ma w artykule-podałem losowe)

# Podstawienie wartości do zmiennych symbolicznych

In [22]:
#podstawienie wartości stałych współczynników
tmp = sp.Matrix([[a_ship,b_ship,m,m3,Cu,Cv,Cr,Vw,d1f,d2f,d3f,d1c,d2c,d3c,k1,k2,k3,a,k1b,k2b,k3b,k4b,k5b]]).subs(params)
a_ship,b_ship,m,m3,Cu,Cv,Cr,Vw,d1f,d2f,d3f,d1c,d2c,d3c,k1,k2,k3,a,k1b,k2b,k3b,k4b,k5b = tmp
#podstawienie wartości, które są obliczane na podstawie stałych współczynników
#tmp2 = sp.Matrix([[k_r,k_th,d10,d20,d30,delta_1,delta_2,delta_3,th3c,th3f]]).subs(params)
#k_r,k_th,d10,d20,d30,delta_1,delta_2,delta_3,th3c,th3f = tmp2
k_th = k1 * k2
k_r = k1 + k2 - d3f
th3f = a/((k3 - k1)*(k3 - k2))
th3c = a/(k3**2 - (d3c + k_r)*k3 + k1*k2)
d10 = (d1c + d1f)/2
d20 = (d2c + d2f)/2
d30 = (d3c + d3f)/2
th30 = (th3c + th3f)/2
delta_1 = (d1c - d1f)/2
delta_2 = (d2c - d2f)/2
delta_3 = (th3c - th3f)/2
Delta_1 = delta_1 #wartości wielkich delta przyjęto równe małym deltom 
Delta_2 = delta_2
Delta_3 = delta_3
d1 = d10 + Delta_1
d2 = d20 + Delta_2
d3 = d30 + Delta_3#
th3 = th30 + Delta_3
delta_d = d3 - d3f
lam_1 = ((delta_d + k1 + k2) + sp.sqrt(delta_d**2 + 2*delta_d*(k1 + k2) + (k1 - k2)**2))/2
lam_2 = ((delta_d + k1 + k2) - sp.sqrt(delta_d**2 + 2*delta_d*(k1 + k2) + (k1 - k2)**2))/2
th_0=1 # wartość początkowa kąta theta
r_0=0  # wartość początkowa prędkości kątowej r
th_1 = ((lam_2*th_0 + r_0)/(lam_2 - lam_1)) + a/((lam_2 - lam_1)*(k3 - lam_1))
th_2 = ((k1*th_0 + r_0)/(lam_1 - lam_2)) + a/((lam_1 - lam_2)*(k3 - lam_2))
th_3 = a/((lam_1 - k3)*(lam_2 - k3))
rf = R
af = Alp
Ff = F
tta = Th# ???funkcja Th czy kąt theta
b2 = sp.Matrix([[1/m,0,0,0,0]]).T # macierz sterowania

## D